#**<font size="10" color='cornflowerblue'><ins>Welcome to exercise 9**
## **Note**: After choosing or changing an answer, run its block to save changes.
## **Note**: It is best to avoid Runtime -> Run all.



Block 1#


In [ ]:
#@title #**What is your ID and group number? (<font color="red">*Mandatory</font>)**


def initialize_colab():
  # Verify credentials
  if id_number < 1000 or group_number == 0 or group_number > 1000:
    raise Exception("Are you sure these details are correct?") 
  # Messages to user
  print("ID number saved: ", id_number)
  print("Group number saved: ", group_number)
  print("\nYou may now begin, good luck!")


try:
  # Imports and library initialization
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import numpy as np
  import matplotlib.pyplot as plt
  import seaborn as sns
  import pandas as pd
  from math import sqrt
  from scipy import stats
  import statsmodels.api as sm
  from ipywidgets import interact
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures
  sns.set(style = "white", palette = "muted", color_codes = True)
  sns.set(rc = {'figure.figsize': (10, 10)});
  sns.set_context("talk")

  # Setting up global colab variables
  exercise_number = 9
  executed_blocks = {0, 1}
  block_mapping = {2: 1, 3: 2, 4: 3, 5: 4, 7: 5.1, 9: 5.2, 10: "5.3.1", 
                   11: "5.3.2", 12: "5.3.3", 13: "5.3.4", 14: "5.4",
                   16: "6.1", 17: "6.2", 18: "6.3", 19: "6.4", 21: "7.1", 
                   23: "7.2", 24: "7.3"}
  answers = {}
  data = {}
  id_number =  0#@param {type:"integer"}
  group_number = 0 #@param {type:"integer"}
  initialize_colab()
  initialized = True
  
except Exception as e:
  print(str(e))


# Dependency check function
def verify_dependencies(block_id, required_blocks_indices=[]):
  for block_index in required_blocks_indices:
    if block_index not in executed_blocks:
      raise Exception("You must first run block "+str(block_index)+"#")


# **********************  Statistics helper functions  **********************
def t_fun(deg_freedom,deg_freedom_extra,t_score):
  pval = (stats.t.sf((t_score), deg_freedom))  # two-sided pvalue = Prob(abs(t)>tt)
  if (t_score<0):
    pval=1-pval #
  print ("t(",deg_freedom ,") = ", t_score,", p-value:", pval)
  ttt = stats.t.rvs(deg_freedom,size=5000)
  plot_dist(ttt, height=5,bins=20,color="orange", fit="t")
  plt.axvline(t_score,color="black",linestyle="--",label="Chosen value")
  plt.title("t-distribution")


def F_fun(df1,df2,F_score):
  df1 = float(df1) ##k-1
  df2 =float(df2) ##N-k
  F_score = float(F_score)  ##MSB/MSW
  pval = (stats.f.sf(F_score, df1,df2))  # two-sided pvalue = Prob(abs(t)>tt)
  print("F(",df1,",",df2 ,") = ", F_score,", p-value:", round(pval,3),"\n")
  fff = stats.f.rvs(df1,df2,size=1000)
  sns.distplot(fff,hist=1,kde=0,axlabel="F-score",fit=stats.f,fit_kws={"color":"purple"})
  plt.xlim(0,max(10,F_score*1.1))
  plt.axvline(F_score,color="black",linestyle="--")
  plt.title("F-distribution")


def plot_dist(dist, bins="auto", kde=False, fit="None", color="b", stat="density",height=8, aspect=1):
  plt.figure(0);
  sns.displot(dist, kde=kde,bins=bins,stat=stat,color=color,height=height, aspect=aspect);
  if (fit=="Normal") & (stat=="density"):  
    d = stats.norm.fit(dist)
    mu = d[0]; sigma = d[1]
    x = np.linspace(mu - 3.5*sigma, mu + 3.5*sigma, 200) #limits of dist
    plt.plot(x, stats.norm.pdf(x, mu, sigma),color="black");
  elif (fit=="F") & (stat=="density"):  
    d = stats.f.fit(dist)
    dfn = d[0]; dfd = d[1]
    interval = stats.f.interval(.995,dfn,dfd) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.f.pdf(x, dfn, dfd),color="black");
  elif (fit=="gamma") & (stat=="density"):  
    d = stats.gamma.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.gamma.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.gamma.pdf(x, a, loc, scale),color="black");
  elif (fit=="t") & (stat=="density"):  
    d = stats.t.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.t.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.t.pdf(x, a, loc, scale),color="black");


def perform_fdr_t(t_vals, deg_freedom, alpha):
  old_t=t_vals.copy()
  t_vals=np.sort(t_vals)[::-1]
  pvals = [(stats.t.sf(np.abs(t_score), deg_freedom)) for t_score in t_vals]
  sig=0
  for i in range(len(pvals)):
    if pvals[i]<(alpha*(i+1)/len(t_vals)):
      sig=pvals[i]
  return (sig)


def boot_dist(dist, B=10000,replacement = True):
  sample_matrix = np.random.choice(dist,size=(len(dist),B),replace=replacement)
  means_dist = np.mean(sample_matrix,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist)


def boot_dist_F(data,IV,DV, B=10000, replacement = True):
  levels = np.unique(data[IV])
  F_vals=[]
  for i in range(B):
    samples = [np.random.choice(data[data[IV]==level][DV],size=len(data[data[IV]==level][DV]),
                                replace=replacement) for level in levels]
    f,_=stats.f_oneway(*samples) # average each column resulting in len(dist) X 1 vector
    F_vals.append(f) 
  return (F_vals)


def cohen_d(x,y):
  nx = len(x)
  ny = len(y)
  dof = nx + ny - 2
  return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


def plot_reg_results(x,y):
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
  term="+"*(intercept>0)
  if p_value<0.001: p_value=0.001
  pred_y = r"$\tildey$"
  reg_label = (f"{pred_y}={str(np.round(slope,1))}x{term}{str(np.round(intercept,1))}, p<{np.round(p_value,3)}")
  plt.figure(figsize=(7,7));
  sns.regplot(x=x,y=y);
  plt.title(reg_label);
  return ([slope,intercept,p_value])


def r_score(x, y):
    return round(stats.pearsonr(x, y)[0],3)


def plot_reg_and_minimize(betax,beta0,X,Y):
  student_x = X
  student_y = betax*student_x+beta0
  plt.figure(figsize=(12,8))
  sns.regplot( x=X,y=Y,fit_reg=False, color = "#154318")
  sns.lineplot(x=student_x, y=student_y, color = "r",label=("y="+str(betax)+"x+"+str(beta0)) )
  plt.xlabel('x', color='#1C2833')
  plt.ylabel('y', color='#1C2833')
  plt.ylim(min(Y)*.95,max(Y)*1.05)
  plt.xlim(min(X)*.95,max(X)*1.05)
  SSerror = np.round(np.sum((Y-student_y)**2))
  SST=np.round(np.sum((Y-Y.mean())**2))
  plt.legend(loc='upper left')
  plt.title(f"Your graph:\nSSE ={str(int(SSerror))}, SST={SST}")
  slope = (r_score(X,Y)*np.std(Y)/np.std(X))


def spearman_score(x, y):
    return round(stats.spearmanr(x, y)[0],3)


# Helper Class
class Polynomial(list): 
    def __repr__(self):
        # joiner[first, negative] = str
        joiner = {
            (True, True): '-',
            (True, False): '',
            (False, True): ' - ',
            (False, False): ' + '
        }
 
        result = []
        for power, coeff in reversed(list(enumerate(self))):
            j = joiner[not result, coeff < 0]
            coeff = abs(coeff)
            if coeff == 1 and power != 0:
                coeff = ''
 
            f = {0: '{}{}', 1: '{}{}x'}.get(power, '{}{}x^{}')
 
            result.append(f.format(j, coeff, power))
 
        return ''.join(result) or '0'

Block 2#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1</ins>** 
#@markdown #A researcher used a linear regression model to test if years of marriage can predict happiness in a sample of 40 people. The average happiness level in the sample was 85. He got the following equation for his regression line: 
#@markdown #Y = 5x+10
#@markdown #Then he realized that he forgot 2 participants. One of them was married for 10 years and reported a level of happiness equal to 60. Another one was married for 20 years, and his level of happiness was 110. How adding these 2 participants would affect SSE and SST? 
#@markdown #**How adding these 2 participants would affect SSE and SST?**

block_id = 2
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block2():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "SSE will get larger, SST will get larger", "SSE will get larger, SST will not change", "SSE will not change, SST will get larger", "SSE will not change, SST will not change"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 3#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 2</ins>** 
#@markdown #**Why do we need a linear regression?**

block_id = 3
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block3():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "To test if a Pearson correlation is different than zero", "To build a model which is capable of predicting y values for x values, when they are not in the sample", "To overcome the interpolation problem of judging a correlation from the range of our sample", "To calculate a slope in an elegant and simple equation that represents our model"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 4#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 3</ins>** 
#@markdown #In the following graph you have the correlation of 3 different samples between miles per gallon (MPG) and the weight of the car.
#@markdown <img src="https://drive.google.com/uc?export=view&id=143wvg_DIEhvNKbY5mJcOZ13qQGtZxeQF"></img>
#@markdown #**Which equation matches which line?**

block_id = 4
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block4():
  green_line = 'Choose the correct answer' #@param ["Choose the correct answer", "MPG = 0.011 * Weight + 42.5", "MPG = -0.0068 * Weight + 32.5", "MPG = -0.011 * Weight + 42.5", "MPG = -0.006 * Weight + 27.5", "MPG = 0.006 * Weight + 32.5", "MPG = 0.0068 * Weight + 27.5"]
  if green_line == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  red_line = 'Choose the correct answer' #@param ["Choose the correct answer", "MPG = 0.011 * Weight + 42.5", "MPG = -0.0068 * Weight + 32.5", "MPG = -0.011 * Weight + 42.5", "MPG = -0.006 * Weight + 27.5", "MPG = 0.006 * Weight + 32.5", "MPG = 0.0068 * Weight + 27.5"]
  if red_line == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  blue_line = 'Choose the correct answer' #@param ["Choose the correct answer", "MPG = 0.011 * Weight + 42.5", "MPG = -0.0068 * Weight + 32.5", "MPG = -0.011 * Weight + 42.5", "MPG = -0.006 * Weight + 27.5", "MPG = 0.006 * Weight + 32.5", "MPG = 0.0068 * Weight + 27.5"]
  if blue_line == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = (green_line, red_line, blue_line)
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 5#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4</ins>** 
#@markdown #**Which of the following is true for the equation of the regression line?**

block_id = 5
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block5():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The equation for predicting X from Y would be the same equation for predicting Y from X.", "Even a very high Pearson correlation can produce a very low slope, and even a very low pearson correlation can produce a very high slope (in absolute value)", "It always allows us to accurately predict the dependent variable", "The regression line gives great fits also to non-linear relations."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 6#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5</ins>** 
#@markdown #Below is a data set of 20 participants which shows the relation between the **amount of time** (in hours per day) spent on social media and the **final grade** in the course “Psychology of fairies and other distributions”. Rim wants to know if the amount of time invested in social media can predict the course's final grade. 

block_id = 6
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block6():
  from IPython.display import HTML
  np.random.seed(group_number)
  sd=5
  avgtime=9
  n=20
  time_51 = np.round(np.random.normal(avgtime,sd,n),1) ; time_51[time_51<0]=0
  grade =  np.round(100-2*time_51+np.random.normal(scale=sd,size=n),1); grade[grade>100]=100
  data_q51 = pd.DataFrame({"subject":np.arange(n)+1,"social_media_time":time_51,"final_grade":grade})
  data_q51.index+=1
  return data_q51


try:
  prepare_block()
  data_q51 = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


from IPython.display import HTML
HTML(data_q51.to_html(index=False))

Block 7#

In [ ]:
#@title #**<font size="6" color='cornflowerblue'><ins>Question 5.1</ins>** 
#@markdown #**Which of the following will you use as the dependent variable and which as the independent variable?**

block_id = 7
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)

def block7():
  dependent_variable = "Choose a variable" #@param ["Choose a variable","subject","social_media_time", "final_grade"]
  if dependent_variable == 'Choose a variable':
    raise Exception("You must choose a dependant variable")

  independent_variable = "Choose a variable" #@param ["Choose a variable","subject","social_media_time", "final_grade"]
  if independent_variable == 'Choose a variable':
    raise Exception("You must choose an independant variable")

  answer = (dependent_variable, independent_variable)
  answers[block_id] = answer
  print("Your Answers are: ")
  for ans in answers[block_id]:
    print(ans)
  return dependent_variable, independent_variable

try:
  prepare_block()
  DV51, IV51 = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 8#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins></ins>** 
#@markdown #By building our linear regression model we are trying to find the best line that will fit our data. We measure the goodness of fit by the errors between our line and the data points. Below is a plot of the data and the fitted line. Change the values of the line’s slope to find the best line which has the minimal fit error. 
#@markdown - Hint: you can evaluate your process by evaluating the SSE (the mistake in model's prediction) of each line that you created. 

block_id = 8
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block8():
  slp_g, intcpt_g, rva_g, pva_g, std_err_g = stats.linregress(data_q51[IV51],data_q51[DV51])
  rsq_g = rva_g**2
  beta0 = np.round(intcpt_g,1) 
  #@markdown Set slope
  beta1 =  -8 #@param {type:"slider",min:-8,max:4,step:0.1}
  plot_reg_and_minimize(beta1,beta0,np.array(data_q51[IV51]),np.array(data_q51[DV51]))
  data['slp_g'] = slp_g
  data['rsq_g'] = rsq_g
  return slp_g, rsq_g


try:
  prepare_block()
  slp_g, rsq_g = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 9#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.2</ins>** 
#@markdown #**What are the optimal slope and the corresponding R-squared of the model?**

block_id = 9
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block9():
  slope =  0#@param {type:"number"}
  r_squared =  0#@param {type:"number"} 
  answers[block_id] = (slope, r_squared)
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 10#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.3.1</ins>** 
#@markdown #Now that you matched a regression line to the data, do the opposite – create data points for a specific regression line. Create the following regression lines out of 10 data points – y=x+3, and y = 0.1*x-5. 
#@markdown - **Choose the dots such that for each line you get one instance of p<.001, and one instance with p>.05.**

#@markdown - Try to get the slope and intercept within a margin of 50% from what you are requested - for a requested slope of 2, we will accept a set of Y values that creates a line with a slope between 1-3.  

#@markdown - Write the Y values with commas between them. The first value you write should match x=1, the second should match x=2, until 10


#@markdown x: 1,2,3,4,5,6,7,8,9,10
block_id = 10
required_blocks = range(6, block_id)
warnings.simplefilter(action='ignore', category=DeprecationWarning)


def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block10():
  #@markdown  #**Predicted y=x+3, p<0.001**
  x_a = "1,2,3,4,5,6,7,8,9,10"
  y_a = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  if len(y_a.split(',')) != 10:
    raise Exception("You must insert 10 y values")
  x_a= eval(f"[{x_a}]")
  y_a= eval(f"[{y_a}]")
  a53=plot_reg_results(x_a,y_a)
  a53[0:1] = np.round(a53[0:1],1)
  answers[block_id] = y_a
  data['a53'] = a53
  print("Your answer:", answers[block_id], end = '\n\n')


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 11#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.3.2</ins>** 
#@markdown #**y=x+3, p>0.05**

block_id = 11
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block11():
  x_b = "1,2,3,4,5,6,7,8,9,10"
  y_b = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  if len(y_b.split(',')) != 10:
    raise Exception("You must insert 10 y values")
  x_b= eval(f"[{x_b}]")
  y_b= eval(f"[{y_b}]")
  b53=plot_reg_results(x_b,y_b)
  b53[0:1] = np.round(b53[0:1],1)
  answers[block_id] = y_b
  data['b53'] = b53
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 12#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.3.3</ins>** 
#@markdown #**y= 0.1*x-5, p<0.001**

block_id = 12
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block12():
  x_c = "1,2,3,4,5,6,7,8,9,10"
  y_c = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  if len(y_c.split(',')) != 10:
    raise Exception("You must insert 10 y values")
  x_c= eval(f"[{x_c}]")
  y_c= eval(f"[{y_c}]")
  c53=plot_reg_results(x_c,y_c)
  c53[0:1] = np.round(c53[0:1],1)
  answers[block_id] = y_c
  data['c53'] = c53
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 13#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.3.4</ins>** 
#@markdown #**y = 0.1*x-5, p>0.05**

block_id = 13
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block13():
  x_d = "1,2,3,4,5,6,7,8,9,10"
  y_d = "0,0,0,0,0,0,0,0,0,0" #@param {type:"string"}
  if len(y_d.split(',')) != 10:
    raise Exception("You must insert 10 y values")
  x_d= eval(f"[{x_d}]")
  y_d= eval(f"[{y_d}]")
  d53=plot_reg_results(x_d,y_d)
  d53[0:1] = np.round(d53[0:1],1)
  answers[block_id] = y_d
  data['d53'] = d53
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 14#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.4</ins>** 
#@markdown #**What is the relation between the size of the slope and the p-value?**

block_id = 14
required_blocks = range(6, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block14():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The higher is the slope the lower is the p-value", "The lower is the slope the lower is the p-value", "If I know the size of the slope and the number of points, I can know the p-value.", "I cannot know the p-value just from the size of the slope."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 15#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6</ins>** 
#@markdown #A researcher decided to conduct an experiment in order to test the relation between stress levels (Measuring levels of cortisol ranging from 0-25 mcg/DL) and cognitive function (Scores between 0-100). He hypothesized that stress levels would predict cognitive performance such that high levels of stress will be linked to a reduction in performance in various cognitive tasks. We will build a regression model to decide whether stress levels can be a good predictor of cognitive functions. Your sample size will be n=50.
#@markdown #**Show me the data!**

block_id = 15
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block15():
  np.random.seed(int(group_number+12))
  n61=50
  x=np.round(np.random.normal(50,10,n61))
  y=np.round(-.5*x + np.random.normal(90,17,n61))
  x[x<0] = 0
  y[y<0] = 0
  x[x>100] = 100
  y[y>100] = 100
  pval_61 =np.round(stats.linregress(x,y)[3],4)
  slp_61 =np.round(stats.linregress(x,y)[0],3)
  intcpt_61 =np.round(stats.linregress(x,y)[1],3)
  cor_61 =np.round(stats.linregress(x,y)[2],3)
  print(f"stress levels: {x}\n")
  print(f"cognitive performance: {y}\n")
  print(f"N={n61}, \033[1mpearson correlation={cor_61}, beta0={intcpt_61}, beta1={slp_61}, p={pval_61}")
  data['slp_61'] = slp_61
  return x, y, slp_61, cor_61, intcpt_61

try:
  prepare_block()
  x, y, slp_61, cor_61, intcpt_61 = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 16#

In [ ]:
#@title #**<font size="6" color='cornflowerblue'><ins>Question 6.1</ins>** 
#@markdown #**Which of the following can be used to test if our model effectively predicts cognitive performances?**

block_id = 16
required_blocks = range(15, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)

def block16():
  # Get LOCAL answer from user
  #@markdown &nbsp;&nbsp;&nbsp;&nbsp; option1: Permute the data of the dependent and independent variables, calculate the slope of each permutation, repeat 10,000 times, create a distribution of the results of all permutated slopes, and then count how many slopes in the permuted distribution of the slopes are bigger (smaller) than our original data slope
  option1 = False #@param {type:"boolean"}
  #@markdown &nbsp;&nbsp;&nbsp;&nbsp; option2: Perform a random sampling with replacement of the pairs of data-points calculate the correlation between the distribution of X and the distribution of Y, repeat the process 10000 times, and then test how many correlations in the distribution are below/above zero (depending on the real data correlation).
  option2= False #@param {type:"boolean"}
  #@markdown &nbsp;&nbsp;&nbsp;&nbsp; option3: Calculate the correlation between the variable and use the t-test with the t-distribution to calculate the significance of the correlation coefficient we found in our sample.
  option3= False #@param {type:"boolean"}
  answer = (option1, option2, option3)
  answers[block_id] = answer
  print("Your Answers are: ")
  for ans in answers[block_id]:
    print(ans)

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 17#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.2</ins>** 
#@markdown #**Another way to test our regression model is to make permutations to our data. Choose the correct order of steps that will produce an estimation of the distribution of slopes according to H0.**
#@markdown #**There is only one correct answer.**

block_id = 17
required_blocks = range(15, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block17():
  B=10000
  def samp_replacement(x,y,a=None,b=None,c=None):
    samples = np.random.choice(np.arange(len(x)),len(x),replace=True)
    return (x[samples],y[samples])

  def samp_replacement_x_only(x,y,a=None,b=None,c=None):
    samples = np.random.choice(np.arange(len(x)),len(x),replace=True)
    return (x[samples],y)

  def shuffle_values(x,y,a=None,b=None,c=None):
    samples_x = np.random.choice(np.arange(len(x)),len(x),replace=False)
    samples_y = np.random.choice(np.arange(len(x)),len(x),replace=False)
    return (x[samples_x],y[samples_y])

  def calculate_beta1(x,y,a=None,b=None,c=None):
    return (stats.linregress(x,y)[0])

  def calculate_beta0(x,y,a=None,b=None,c=None):
    return (stats.linregress(x,y)[1])

  def calculate_correlation(x,y,a=None,b=None,c=None):
    return (stats.linregress(x,y)[2])

  def compare_means(x,y):
    return np.mean(x-y)

  def repeat_previous_two(fun1,fun2,x,y,B=B,c=None):
    correl_vec = []
    np.random.seed(1) #should be 1 not group number
    for i in range(B):
      pairs=fun1(x,y)
      curr_cor=fun2(pairs[0],pairs[1])
      correl_vec.append(curr_cor)
    return (correl_vec)

  def create_dist(fun1,fun2,fun3,x,y):
    correlations = fun3(fun1,fun2,x,y) 
    plt.figure(figsize=(7,7)) 
    sns.histplot(correlations, bins=30,color="darkgreen",stat="count")
    sns.kdeplot(correlations,color="black")
    plt.xlabel("calculated values")
    return correlations


  step_dict = {"Sample with replacement from the X values.":samp_replacement_x_only,
                "Compare the mean of the X scores with the mean of the Y scores and save it":compare_means,
                "Sample with replacement from the pairs":samp_replacement,
                "calculate the linear regression fit and save the slope (beta1)":calculate_beta1,
                "calculate the linear fit and save the interception point with the y-axis (beta0)":calculate_beta0,
                "Calculate and save the correlation between x and y.":calculate_correlation,
                "Repeat the previous steps 10,000 times.":repeat_previous_two,
                "Mix the Y values between the different X values.":shuffle_values,
                "create a distribution of all the calculated values that were saved":create_dist}

  step_1a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","Create a distribution of all the datapoints that were saved."]
  if step_1a == 'X':
    raise Exception("You must choose step_1.")
  step_2a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  step_3a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  step_4a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  step_5a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  step_6a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  step_7a = "X" #@param ["X","Sample with replacement from the X values.","Compare the mean of the X scores with the mean of the Y scores and save it","Sample with replacement from the pairs","calculate the linear regression fit and save the slope (beta1)","calculate the linear fit and save the interception point with the y-axis (beta0)","Calculate and save the correlation between x and y.","Repeat the previous steps 10,000 times.","Mix the Y values between the different X values.","create a distribution of all the calculated values that were saved"]
  steps = [step_dict[step_1a],step_dict[step_2a],step_dict[step_3a],step_dict[step_4a]]#by their choice


  try:
    dist61=np.array(steps[3](step_dict[step_1a],step_dict[step_2a],step_dict[step_3a],x,y) )
    np.save('dist.npy', dist61)
  except:
    if step_4a ==  "create a distribution of all the calculated values that were saved":
      if step_3a == "Repeat the previous two steps 10,000 times.":
        plt.figure(figsize=(7,7)) 
        plt.hist(np.random.normal(loc=.7,scale=.1,size=10000),bins=30, color="orange")
      else:
        plt.figure(figsize=(7,7)) 
        plt.hist(np.repeat(slp_61,10000),bins=30, color="orange")
      if step_4a !=  "create a distribution of all the calculated values that were saved":
        print("You didn't plot anything.")
  plt.title("Your distribution");
  answers[block_id] = step_1a, step_2a, step_3a, step_4a, step_5a, step_6a, step_7a 
  print("Your answer:", answers[block_id])
  return dist61


try:
  prepare_block()
  dist61 = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 18#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.3</ins>** 
#@markdown #**Now that you have created the distribution of the permutations, what do you need to get from it in order to calculate the p-value on the regression line’s slope?**
#@markdown - Note: Once you filled in your answer, you will get below the number of observations that you decided to count.
block_id = 18
required_blocks = range(15, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block18():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The number of observations smaller than 0.", "The number of observations greater than 0.", "The number of observations equal or above our sample’s slope", "The number of observations equal or below our sample’s slope"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  output_dict = {"Choose an answer.":"You must choose an answer.",
                "The number of observations smaller than 0.":np.sum(dist61<0),
                "The number of observations greater than 0.":np.sum(dist61>0),
                "The number of observations equal or above our sample’s slope":np.sum(dist61>slp_61),
                "The number of observations equal or below our sample’s slope":np.sum(dist61<slp_61)}

  print(f"\nThe result is: {output_dict[Answer]}\n")
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 19#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.4</ins>** 
#@markdown #**What is your p-value on the permutation distribution?**
block_id = 19
required_blocks = range(15, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block19():
  p_value = 0 #@param {type:"number"}  if Answer == 'Choose the correct answer':
  answers[block_id] = p_value
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 20#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7</ins>** 
#@markdown #The blocks below present a dataset of 15 points you sampled for your study. First, find the model that fits best these 15 points.
#@markdown #**Your task is to choose which model fits best the training set and which model fits best the test set.**
#@markdown - for example, 1 is fitting a regular linear regresion line. 3 is finding the a,b,c,d that best fits ax^3+bx^2+cx+d to the data  

block_id = 20
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block20():
  def plot_reg_poly_crossval(degree,x,y,training_set_size):
    line = np.linspace(min(x),max(x),100)
    x_test = x[training_set_size:]
    y_test = y[training_set_size:]
    x=x[:training_set_size]
    y=y[:training_set_size]
    plt.figure(figsize=(7,7));
    sns.regplot(x=x,y=y, fit_reg=False, color='black')
    
    #get coefficients just for plot
    coefs = np.round(np.polyfit(x,y, degree),2)
    #model_prediction = np.sum([coefs[-i-1]*line**(i) for i in range(len(coefs))],axis=0)
    #model_prediction_y = np.sum([coefs[-i-1]*x**(i) for i in range(len(coefs))],axis=0)
    
  
    polynomial_features= PolynomialFeatures(degree=degree)
    #format data for polynomial regression
    x_c = x[:, np.newaxis]
    line_c = line[:, np.newaxis]
    y_c = y[:, np.newaxis]
    x_test_c = x_test[:,np.newaxis]
    x_test_poly = polynomial_features.fit_transform(x_test_c)
    x_poly = polynomial_features.fit_transform(x_c)
    line_poly = polynomial_features.fit_transform(line_c)
  
    #fit
    model = LinearRegression()
    model.fit(x_poly, y_c)
    
    #predict on test set and creat line
    y_poly_pred = model.predict(x_poly); y_poly_pred = [i[0] for i in y_poly_pred]
    y_test_poly_pred = model.predict(x_test_poly); y_test_poly_pred = [i[0] for i in y_test_poly_pred]
    line_poly_pred = model.predict(line_poly)
    
    #evaluate model and CV
    SSE=np.sum((y-y_poly_pred)**2)
    SSE_test = np.sum((y_test-y_test_poly_pred)**2)
    SST=np.sum((y-y.mean())**2)
    SST_test = np.sum((y_test-y_test.mean())**2)
  
    plt.plot(line,line_poly_pred)
    
    reg_label = Polynomial(coefs)
    print(f"SSE = {int(SSE)}, SST= {int(SST)}")
    plt.title("Best fit: "+str(reg_label))
    
    return (reg_label,line,line_poly_pred,y_test,x_test,SSE_test,SST_test,SSE,SST)
  
  
  degree ="Choose an answer" #@param ["Choose an answer","1","2","3","5","7"]
  degree=int(degree)
  
  np.random.seed(1)
  
  slp_7_x2 = 4; 
  slp_7_x = 2
  slp_7_intcpt = 5
  n7=30
  training_set_size=15

  x7 = np.random.uniform(0,6,n7)
  y7 = slp_7_x2*x7**2 + slp_7_x*x7 + slp_7_intcpt+np.random.normal(scale=20,size=n7)
  reg_label,line,line_poly_pred,y_test,x_test,SSE_test,SST_test,SSE,SST = plot_reg_poly_crossval(degree,x7,y7,training_set_size)
  return slp_7_x2, slp_7_x, slp_7_intcpt, x7, y7, training_set_size


try:
  prepare_block()
  slp_7_x2, slp_7_x, slp_7_intcpt, x7, y7, training_set_size = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 21#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.1</ins>** 
#@markdown #**Which degree of polynomial fit gives you the best fit?**

block_id = 21
required_blocks = range(20, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block21():
  Answer ="Choose an answer" #@param ["Choose an answer", "1", "2", "3", "5", "7"]
  if Answer == 'Choose an answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 22#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins></ins>** 
#@markdown #The blocks below present a dataset of another 15 points you sampled for your study. They come from the same group you sampled before so should have the same model. Therefore, you want to check how these new points fit the previous models you found based on the previous data. Run the block to see the fitting results for each model.
#@markdown #**Run to test the model fitted to the data before:**

block_id = 22
required_blocks = range(20, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block22():
  def plot_reg_poly_crossval_test(degree,x,y,training_set_size):
    line = np.linspace(min(x),max(x),100)
    x_test = x[training_set_size:]
    y_test = y[training_set_size:]
    x=x[:training_set_size]
    y=y[:training_set_size]
    plt.figure(figsize=(7,7));
    sns.regplot(x=x_test,y=y_test, fit_reg=False, color='black')
    
    #get coefficients just for plot
    coefs = np.round(np.polyfit(x,y, degree),2)
    #model_prediction = np.sum([coefs[-i-1]*line**(i) for i in range(len(coefs))],axis=0)
    #model_prediction_y = np.sum([coefs[-i-1]*x**(i) for i in range(len(coefs))],axis=0)
    
  
    polynomial_features= PolynomialFeatures(degree=degree)
    #format data for polynomial regression
    x_c = x[:, np.newaxis]
    line_c = line[:, np.newaxis]
    y_c = y[:, np.newaxis]
    x_test_c = x_test[:,np.newaxis]
    x_test_poly = polynomial_features.fit_transform(x_test_c)
    x_poly = polynomial_features.fit_transform(x_c)
    line_poly = polynomial_features.fit_transform(line_c)
  
    #fit
    model = LinearRegression()
    model.fit(x_poly, y_c)
    
    #predict on test set and creat line
    y_poly_pred = model.predict(x_poly); y_poly_pred = [i[0] for i in y_poly_pred]
    y_test_poly_pred = model.predict(x_test_poly); y_test_poly_pred = [i[0] for i in y_test_poly_pred]
    line_poly_pred = model.predict(line_poly)
    
    #evaluate model and CV
    SSE=np.sum((y-y_poly_pred)**2)
    SSE_test = np.sum((y_test-y_test_poly_pred)**2)
    SST=np.sum((y-y.mean())**2)
    SST_test = np.sum((y_test-y_test.mean())**2)
    reg_label = Polynomial(coefs)
    
    plt.plot(line,line_poly_pred)
    print(f"SSE of test set = {np.round(SSE_test,2)}, SST of test set= {np.round(SST_test,2)}")
    plt.title("Testing fit on new set: "+str(reg_label));
    return (reg_label,line,line_poly_pred,y_test,x_test,SSE_test,SST_test)


  degree ="Choose an answer" #@param ["Choose an answer","1","2","3","5","7"]
  degree=int(degree)
  
  _ = plot_reg_poly_crossval_test(degree,x7,y7,training_set_size)


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 23#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.2</ins>** 
#@markdown #**Which degree of polynomial fits best in the test data?**

block_id = 23
required_blocks = range(20, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block23():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "1", "2", "3", "5", "7"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 24#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.3</ins>** 
#@markdown #**Did you get the same results for both stages? Why is that?**

block_id = 24
required_blocks = range(20, block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block24():
  Answer = "Choose the correct answer" #@param ["Choose the correct answer", "They are the same because models don't lie", "They are the same because the fitting procedure always calculates the deviations of the points from the model, so the more complex is the model (the higher is the degree) the better fit we'll get for the datapoints we're trying to fit.", "They are different because the more complex is the model (the higher is the degree), the better fit we can get, but it costs us with fitting the noise in the data so new points will deviate stronger from this over-fitted model", "They are different because it's two different samples so I don't expect to get the same model every time."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


<br>
<br>

---

<br>
<br>


**<p><font size="10"><font color='cornflowerblue'> Good job!</ins></p>** </font>

#**You finished answering the questions for this exercise! Here are some important things to consider:**


* You can view your answers by running the next block. 
*After changing an answer to a question, you will need to run the 'Show your answers' again for it to reflect changes.
* <b>If you change an answer to a question, you will need to run it again for changes to be saved. 


Block 25#

In [ ]:
#@title **<font color = cornflowerblue> <font size = 6>Show Your Answers <br>(Run to refresh after saving a question)**

block_id = 25


def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id))


def answers_printer():
  for block in sorted(answers):
    print("Question " + str(block_mapping[block]) + ": " + str(answers[block]))

try:
  prepare_block()
  answers_printer()
  # blocks_run[block_id] = True
except Exception as e:
  print(str(e))


If you are ready to submit your answers, go ahead and press 'Save!'💾 .


This will prompt you to download your answers file. 


After downloading you will need to submit the file to moodle.
Good luck!



Block 26#

In [ ]:
#@title  **<p><font size="10"><font color='cornflowerblue'><ins> <---  💾 SAVE! </ins></p>**
#@markdown If you are ready to submit your answers, go ahead and press 'Save!'💾 . This will prompt you to download your answers file. After downloading you will need to submit the file to moodle. Good luck!

from google.colab import files
from shutil import make_archive
from os import path, remove
from zipfile import ZipFile

block_id = 26

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id- 1))


def answer_saver(file):
  for block in sorted(answers):
    file.write("Question " + str(block_mapping[block]) + ": " + str(answers[block])+'\n')


def submission_saver():
  # Writing answers
  f = open("ex"+str(exercise_number)+"_submission", "w")
  f.write("#"+str(id_number)+"\n")
  f.write("#"+str(group_number)+"\n")
  answer_saver(f) 
  f.write("#End of answers")
  f.close()

  # Writing data
  g = open("ex"+str(exercise_number)+"_data", "w")
  g.write(str(data))
  g.close()
  datazip = ZipFile("ex"+str(exercise_number)+"_data"+'.zip', 'w')
  datazip.write("ex"+str(exercise_number)+"_data")
  datazip.close()

  # Creating submission file
  zipObj = ZipFile("ex"+str(exercise_number)+'.zip', 'w')
  zipObj.write("ex"+str(exercise_number)+'_submission')
  zipObj.write("ex"+str(exercise_number)+"_data")
  zipObj.write("dist.npy")
  
  zipObj.close()
  files.download("ex"+str(exercise_number)+'.zip') 

  # Message to user
  print("Well done! You are now downloading your submission file")


try:
  prepare_block()
  submission_saver()
except Exception as e:
  print(str(e))

